## 동전을 인식하고 그 총액을 계산하는 프로그램

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/kdt_231026/m8_vision

/content/drive/MyDrive/kdt_231026/m8_vision


In [3]:
from google.colab.patches import cv2_imshow

In [ ]:
!pwd

/content/drive/MyDrive/kdt_231026/m8_vision


In [4]:
!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.4/312.4 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 113.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 10.7 MB/s eta 0:00:00


In [6]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import gradio as gr
import tempfile

# 이미지 전처리 함수
def preprocessing(image):
    if image is None:
        return None, None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 2, 2)
    _, th_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    mask = np.ones((3, 3), np.uint8)
    th_img = cv2.morphologyEx(th_img, cv2.MORPH_OPEN, mask)

    return image, th_img

def find_coins(image):
    # cv2.findContours 함수는 이미지에서 윤곽선을 찾는 데 사용.
    # cv2.RETR_EXTERNAL: 이 옵션은 이미지의 가장 바깥쪽 윤곽선만을 검출. 이는 각 동전의 외곽선만을 찾는 데 적합합니다.
    # cv2.CHAIN_APPROX_SIMPLE: 이 근사 방법은 윤곽선을 구성하는 점들 중 필요한 점들만을 반환하여 메모리를 절약. 예를 들어, 직사각형의 경우 4개의 모서리 점만을 반환.
    results = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # OpenCV 4.x 이상에서 results[0]은 윤곽선 리스트, results[1]은 계층 정보
    # OpenCV 3.x에서 results[0]은 수정된 원본 이미지, results[1]은 윤곽선 리스트
    contours = results[0] if int(cv2.__version__[0]) >= 4 else results[1] # 윤곽선 리스트를 선택하기 위한 조건문

    # 리스트 생성 방식
    circles = [cv2.minEnclosingCircle(c) for c in contours] # 주어진 점들의 집합(여기서는 윤곽선 c)을 모두 포함하는 가장 작은 원을 찾습니다
    circles = [(tuple(map(int, center)), int(radius)) # 원의 중심 좌표와 반지름을 정수형으로 변환
               for center, radius in circles if radius>25] # 각 원의 반지름이 25 픽셀보다 큰 경우에만 해당 원을 새로운 리스트에 포함
    return [(tuple(map(int, center)), int(radius)) for center, radius in circles]

# 동전 이미지 만들기 함수
def make_coin_img(src, circles):
    coins = []
    for center, radius in circles:
        r = radius * 3
        mask = np.zeros((r, r, 3), np.uint8)
        cv2.circle(mask, (r // 2, r // 2), radius, (255, 255, 255), cv2.FILLED)
        coin = cv2.getRectSubPix(src, (r, r), center)
        coin = cv2.bitwise_and(coin, mask)
        coins.append(coin)
    return coins

# 동전 히스토그램 계산 함수
def calc_histo_hue(coin):
    hsv = cv2.cvtColor(coin, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0], None, [32], [0, 180])
    return hist.flatten()

# 동전 그룹화 함수
def grouping(hists):
    ws = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 8, 6, 5, 4, 3, 2, 1, 0, 0, 0, 0, 0, 0]
    sim = np.multiply(hists, ws)
    similaritys = np.sum(sim, axis=1) / np.sum(hists, axis=1)
    return [1 if s > 1.2 else 0 for s in similaritys]

# 동전 분류 함수 : 동전의 그룹(groups 값)과 반지름 크기에 따라 동전의 종류를 결정
def classify_coins(circles, groups):
    ncoins = [0] * 4
    g = np.full((2,70), -1, np.int32) # np.full((2,70), -1, np.int32)
    g[0, 26:47], g[0, 47:50], g[0, 50:] = 0, 2, 3
    g[1, 36:44], g[1, 44:50], g[1, 50:] = 1, 2, 3
    for group, (_, radius) in zip(groups, circles):
        coin = g[group, radius]
        ncoins[coin] += 1
    return np.array(ncoins)

def put_string(frame, text, pt, value=None, color=(120, 200, 90)) :
    text = str(text) + str(value)
    shade = (pt[0] + 2, pt[1] + 2)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, text, shade, font, 0.7, (0, 0, 0), 2) # 그림자 효과
    cv2.putText(frame, text, pt   , font, 0.7, color, 2) # 작성 문자

# Gradio 웹 인터페이스에 사용될 이미지 처리 및 결과 반환 함수
def process_and_display_coin(image):
    # 임시 파일 생성 없이 바로 numpy 배열로 전처리 함수에 전달
    # image.save(temp_file_name) 대신 아래 코드 사용
    cv2_img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Gradio에서 받은 이미지는 RGB 순서, OpenCV는 BGR 순서를 사용

    # 전처리 함수에 OpenCV 이미지 배열을 직접 전달
    processed_image, th_img = preprocessing(cv2_img)
    if processed_image is None or th_img is None:
        return "Failed to process the image.", "No coins detected."

    circles = find_coins(th_img)
    coin_imgs = make_coin_img(processed_image, circles)
    coin_hists = [calc_histo_hue(coin) for coin in coin_imgs]
    groups = grouping(coin_hists)
    ncoins = classify_coins(circles, groups)

    # 결과 이미지에 동전 위치와 값 표시
    coin_value = np.array([10, 50, 100, 500])
    for i in range(4):
      print("%3d원: %3d개" % (coin_value[i], ncoins[i]))

    total = sum(coin_value * ncoins )           # 동전금액* 동전별 개수
    total_str = "Total coin: {:,} Won".format(total)            # 계산된 금액 문자열
    print(total_str)                                                 # 콘솔창에 출력
    put_string(processed_image, total_str, (650, 50), '', (0,230,0))

    color = [(0, 0, 255), (0, 255, 255), (0, 255, 0), (255, 0, 255)]
    for i, (c, r) in enumerate(circles):
      cv2.circle(processed_image, c, r, color[groups[i]], 2)
      put_string(processed_image, i, (c[0] - 15, c[1] - 10), '', color[2])  # 검출 순번과 동전 반지름 표시
      put_string(processed_image, r, (c[0], c[1] + 15), '', color[3])



    # 처리 결과 이미지를 RGB로 변환하여 반환
    result_image = cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB)

    return result_image, total_str


# Gradio 인터페이스 정의
iface = gr.Interface(fn=process_and_display_coin,
                     inputs=gr.Image(),
                     outputs=[gr.Image(type="numpy", label="Result"), "text"],
                     title="Coin Detector",
                     description="Upload an image of coins to calculate their total value.")

iface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://1e453544a91de81056.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


 10원:   7개
 50원:  19개
100원:   2개
500원:   3개
Total coin: 2,720 Won
 10원:   8개
 50원:   0개
100원:   7개
500원:   5개
Total coin: 3,280 Won
 10원:   8개
 50원:   0개
100원:   7개
500원:   5개
Total coin: 3,280 Won
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1e453544a91de81056.gradio.live
